In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

In [44]:

# Base URL of the webpage
base_url = "https://mediadive.dsmz.de"


# Function to extract data from a single page
def extract_data_from_page(url, page, retries=5):
    params = {"p": page}
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            data = []
            table_rows = soup.find_all('tr')[1:]  # Skip the header row
            for row in table_rows:
                columns = row.find_all('td')
                row_data = [page]  # Add the page number

                # Name and Name Link
                name_tag = columns[0].find('a')
                name = name_tag.get_text(strip=True)
                name_link = base_url + name_tag['href']
                row_data.append(name)
                row_data.append(name_link)

                # Volume
                volume = columns[1].get_text(strip=True)
                row_data.append(volume)

                # Number of media
                number_of_media = columns[2].get_text(strip=True)
                row_data.append(number_of_media)

                # First defined in and link
                first_defined_in_tag = columns[3].find('a')
                if first_defined_in_tag:
                    first_defined_in = first_defined_in_tag.get_text(strip=True)
                    first_defined_in_link = base_url + first_defined_in_tag['href']
                    row_data.append(first_defined_in)
                    row_data.append(first_defined_in_link)
                
                if "." in name:
                    if name.split(".")[0] == "Main sol":
                        main_solution = "yes"
                    else:
                        main_solution = "no"
                else:
                    main_solution = "no"
                row_data.append(main_solution)
                
                data.append(row_data)

            return data
        except (requests.exceptions.RequestException, ConnectionResetError) as e:
            print(f"Error fetching page {page}: {e}. Retrying {attempt + 1}/{retries}...")
            time.sleep(1)  # Wait before retrying
    return []


# Main scraping process
all_data = []
num_pages = 285  # Adjust the number of pages you want to scrape

max_workers = 128  # Adjust based on the MacBook M3 Pro capabilities

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_page = {executor.submit(extract_data_from_page, base_url + "/solutions", page): page for page in
                      range(1, num_pages + 1)}

    for future in tqdm(as_completed(future_to_page), total=num_pages, desc="Extracting data from pages"):
        page_data = future.result()
        all_data.extend(page_data)

# Sort the data based on the page number to maintain the order
all_data.sort(key=lambda x: x[0])

# Create a DataFrame from the extracted data
columns = ["Page", "Name", "Name Link", "Volume", "Number of media", "First defined in",
           "First defined in Link", "Main Solution"]
df = pd.DataFrame(all_data, columns=columns)
df.drop(columns=["Page"], inplace=True)  # Remove the page column if not needed

df
# df.to_csv('solutions.csv', index=False)

Extracting data from pages: 100%|██████████| 285/285 [00:11<00:00, 25.33it/s]


,Name,Name Link,Volume,Number of media,First defined in,First defined in Link,Main Solution
0,Main sol. 1,https://mediadive.dsmz.de/solutions/S1?p=1,1000 ml,21,Medium 1,https://mediadive.dsmz.de/medium/1,yes
1,Main sol. 1a,https://mediadive.dsmz.de/solutions/S3?p=1,1000 ml,1,Medium 1a,https://mediadive.dsmz.de/medium/1a,yes
2,Main sol. 2,https://mediadive.dsmz.de/solutions/S4?p=1,1000 ml,1,Medium 2,https://mediadive.dsmz.de/medium/2,yes
3,Main sol. 3,https://mediadive.dsmz.de/solutions/S5?p=1,1000 ml,1,Medium 3,https://mediadive.dsmz.de/medium/3,yes
4,Main sol. 6,https://mediadive.dsmz.de/solutions/S6?p=1,1000 ml,1,Medium 6,https://mediadive.dsmz.de/medium/6,yes
...,...,...,...,...,...,...,...
5678,Biotine,https://mediadive.dsmz.de/solutions/S6407?p=284,,0,Medium 1802,https://mediadive.dsmz.de/medium/1802,no
5679,Vitamine B12,https://mediadive.dsmz.de/solutions/S6408?p=284,,0,Medium 1802,https://mediadive.dsmz.de/medium/1802,no
5680,Biotine,https://mediadive.dsmz.de/solutions/S6409?p=285,,0,Medium 1802,https://mediadive.dsmz.de/medium/1802,no
5681,"NaHCO3, 10%",https://mediadive.dsmz.de/solutions/S6410?p=285,,0,no,None,None


In [47]:
# Function to extract data from a single page
def extract_data_from_namelink(namelink, pbar, retries=5):
    for attempt in range(retries):
        try:
            response = requests.get(namelink)
            response.raise_for_status()
            soup_n = BeautifulSoup(response.content, 'html.parser')
            
            solution_table = soup_n.find('table', class_="table solution-table simple") if soup_n.find('table', class_="table solution-table simple") else None
            compound_id = []
            compound = []
            amount = []
            unit = []
            g_l = []
            mmol_l = []
            steps = []
            if solution_table:
                for row in solution_table.find_all('tr'):
                    if row.find('th'):
                        continue
                    # Check if the row has a 'class' attribute
                    if 'class' not in row.attrs:
                        print("Row without 'class' attribute:", row, namelink)
                    if 'step-row' in row['class']:
                        new_step = row.find('td').get_text(strip=True) if row.find('td') else None
                        steps.append('"'+new_step+'"')
                    else:
                        td = row.find_all('td') if row.find('td') else None
                        if td:
                            new_c_id = td[0].find('a')['href'].split('/')[-1] if td[0].find('a') else None
                            new_c = td[0].find('a').get_text(strip=True) if td[0].find('a') else None
                            new_amount = td[1].get_text(strip=True)
                            new_unit = td[2].find('span').get_text(strip=True) if td[2].find('span') else None
                            new_g_l = td[3].get_text(strip=True)
                            new_mmol_l = td[4].get_text(strip=True)
                        else:
                            new_c_id = None
                            new_c = None
                            new_amount = None
                            new_unit = None
                            print("Row without td:", row, namelink)
                        compound_id.append(new_c_id)
                        if new_c:
                            compound.append('"'+new_c+'"')
                        else:
                            compound.append(new_c)
                        amount.append(new_amount)
                        if new_unit:
                            unit.append('"'+new_unit+'"')
                        else:
                            unit.append(new_unit)
                        g_l.append(new_g_l)
                        mmol_l.append(new_mmol_l)
                        
            col_lg7 = soup_n.find('div', class_="col-lg-7") if soup_n.find('div', class_="col-lg-7") else None
            if col_lg7:
                equipment_div = col_lg7.find('div', class_="p-10") if col_lg7.find('div', class_="p-10") else None
                if equipment_div:
                    equipment = ['"'+span.get_text(strip=True)+'"' for span in equipment_div.find_all('span')] if equipment_div.find('span') else None
                else:
                    equipment = None
            else:
                equipment = None
            
            media_table = soup_n.find('table', class_="table small simple") if soup_n.find('table', class_="table small simple") else None
            media_id = []
            media_name = []
            media_link = []
            if media_table:
                for row in media_table.find_all('tr'):
                    td = row.find_all('td') if row.find('td') else None
                    if td:
                        new_media_id = td[0].find('a').get_text(strip=True) if td[0].find('a') else None
                        new_media_name = td[1].get_text(strip=True)
                        new_media_link = base_url + td[0].find('a')['href'] if td[0].find('a') else None
                    else:
                        new_media_id = None
                        new_media_name = None
                        new_media_link = None
                    media_id.append(new_media_id)
                    if new_media_name:
                        media_name.append('"'+new_media_name+'"')
                    else:
                        media_name.append(new_media_name)
                    media_link.append(new_media_link)
                    
            pbar.update(1)
            return [namelink, compound_id, compound, amount, unit, g_l, mmol_l, steps, equipment, media_id, media_name, media_link]
        except (requests.exceptions.RequestException, ConnectionResetError) as e:
            print(f"Error fetching name link {namelink}: {e}. Retrying {attempt + 1}/{retries}...")
            time.sleep(1)  # Wait before retrying
    return [namelink, None, None, None, None, None, None, None, None, None, None, None]

# Extract additional data from all Name links
namelink_data = []
name_links = df['Name Link'].dropna().unique()

with tqdm(total=len(name_links), desc="Namelink Progress") as pbar:
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_name_link = {executor.submit(extract_data_from_namelink, name_link, pbar): name_link for name_link in name_links}
        for future in as_completed(future_to_name_link):
            link_data = future.result()
            namelink_data.append(link_data)

    namelink_data_columns = ["Name Link", "Compound ID", "Compound", "Amount", "Unit", "Conc. [g/L]", "Conc. [mM]", "Steps", "Equipment", "Media ID with this solution", "Media name with this solution", "Media Link with this solution"]
    namelink_df = pd.DataFrame(namelink_data, columns=namelink_data_columns)
    
    merged_df = pd.merge(df, namelink_df, on='Name Link', how='left').fillna("")

merged_df

Namelink Progress:  12%|█▏        | 655/5683 [00:13<04:43, 17.74it/s]

Error fetching name link https://mediadive.dsmz.de/solutions/S595?p=27: ('Connection broken: IncompleteRead(2042 bytes read, 8198 more expected)', IncompleteRead(2042 bytes read, 8198 more expected)). Retrying 1/5...


Namelink Progress: 100%|██████████| 5683/5683 [01:56<00:00, 48.63it/s] 


,Name,Name Link,Volume,Number of media,First defined in,First defined in Link,Main Solution,Compound ID,Compound,Amount,Unit,Conc. [g/L],Conc. [mM],Steps,Equipment,Media ID with this solution,Media name with this solution,Media Link with this solution
0,Main sol. 1,https://mediadive.dsmz.de/solutions/S1?p=1,1000 ml,21,Medium 1,https://mediadive.dsmz.de/medium/1,yes,"[1, 2, 3, 4]","[""Peptone"", ""Meat extract"", ""Agar"", ""Distilled...","[5, 3, 15, 1000]","[""g"", ""g"", ""g"", ""ml""]","[5, 3, 15, -]","[-, -, -, -]","[""1Adjust pH to7.0."", ""2For Bacillus strains t...",,"[1, 2, 420, 429b, 429a, 438, 457d, 457c, 457a,...","[""NUTRIENT AGAR"", ""BACILLUS PASTEURII MEDIUM"",...","[https://mediadive.dsmz.de/medium/1, https://m..."
1,Main sol. 1a,https://mediadive.dsmz.de/solutions/S3?p=1,1000 ml,1,Medium 1a,https://mediadive.dsmz.de/medium/1a,yes,"[1, 2, 3, 4]","[""Peptone"", ""Meat extract"", ""Agar"", ""Distilled...","[5, 3, 15, 1000]","[""g"", ""g"", ""g"", ""ml""]","[5, 3, 15, -]","[-, -, 44.599, -]","[""1Rehydrate and grow lyophilized cells from t...",,[1a],"[""REACTIVATION WITH LIQUID MEDIUM 1""]",[https://mediadive.dsmz.de/medium/1a]
2,Main sol. 2,https://mediadive.dsmz.de/solutions/S4?p=1,1000 ml,1,Medium 2,https://mediadive.dsmz.de/medium/2,yes,"[1, 339]","[""Medium 1"", ""Urea""]","[1000, 20]","[""ml"", ""g/l""]","[-, 20]","[-, 333.023]","[""1To medium 1 add 20 g/l urea before autoclav...","[""Autoclave""]",[2],"[""BACILLUS PASTEURII MEDIUM""]",[https://mediadive.dsmz.de/medium/2]
3,Main sol. 3,https://mediadive.dsmz.de/solutions/S5?p=1,1000 ml,1,Medium 3,https://mediadive.dsmz.de/medium/3,yes,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 4]","[""Glucose"", ""Mannitol"", ""CaCl2x 2 H2O"", ""MgSO4...","[5.00, 5.00, 0.10, 0.10, 5.00, 0.90, 0.10, 0.0...","[""g"", ""g"", ""g"", ""g"", ""mg"", ""g"", ""g"", ""g"", ""g"",...","[5, 5, 0.1, 0.1, 0.005, 0.9, 0.1, 0.01, 5, 15, -]","[27.753, 27.447, 0.68, 0.406, 0.021, 5.167, 0....","[""1Adjust pH to 7.3."", ""2Sterilize glucose and...",,[3],"[""AZOTOBACTER MEDIUM""]",[https://mediadive.dsmz.de/medium/3]
4,Main sol. 6,https://mediadive.dsmz.de/solutions/S6?p=1,1000 ml,1,Medium 6,https://mediadive.dsmz.de/medium/6,yes,"[10, 11, 8, 7, 12, 14, 15, 3, 4]","[""K2HPO4"", ""KH2PO4"", ""MgSO4x 7 H2O"", ""CaCl2x 2...","[0.80, 0.20, 0.50, 0.05, 0.01, 1.00, 20.00, 15...","[""g"", ""g"", ""g"", ""g"", ""g"", ""mg"", ""g"", ""g"", ""ml""]","[0.8, 0.2, 0.5, 0.05, 0.01, 1.0e-3, 20, 15, -]","[4.593, 1.47, 2.029, 0.34, 0.036, 0.006, 126.4...",[],"[""Autoclave""]",[6],"[""ALLANTOIN MINERAL MEDIUM""]",[https://mediadive.dsmz.de/medium/6]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5678,Biotine,https://mediadive.dsmz.de/solutions/S6407?p=284,,0,Medium 1802,https://mediadive.dsmz.de/medium/1802,no,[],[],[],[],[],[],[],,[],[],[]
5679,Vitamine B12,https://mediadive.dsmz.de/solutions/S6408?p=284,,0,Medium 1802,https://mediadive.dsmz.de/medium/1802,no,[],[],[],[],[],[],[],,[],[],[]
5680,Biotine,https://mediadive.dsmz.de/solutions/S6409?p=285,,0,Medium 1802,https://mediadive.dsmz.de/medium/1802,no,[],[],[],[],[],[],[],,[],[],[]
5681,"NaHCO3, 10%",https://mediadive.dsmz.de/solutions/S6410?p=285,,0,no,,,[],[],[],[],[],[],[],,[],[],[]


In [48]:
merged_df.to_csv("solutions.csv", index=False)

In [ ]:

# 
# # Load the initial data
# df = pd.read_csv('initial_dsmz_data.csv')
# 
# import requests
# from bs4 import BeautifulSoup
# import time
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm import tqdm
# 
# 
# # Function to fetch HTML content and parse it with BeautifulSoup
# def fetch_html_structure(url, retries=5):
#     for attempt in range(retries):
#         try:
#             response = requests.get(url)
#             response.raise_for_status()
#             soup = BeautifulSoup(response.content, 'html.parser')
#             return soup
#         except (requests.exceptions.RequestException, ConnectionResetError) as e:
#             print(f"Error fetching HTML content from {url}: {e}. Retrying {attempt + 1}/{retries}...")
#             time.sleep(5)  # Wait before retrying
#     return None
# 
# 
# # Function to extract key data from the parsed HTML content
# def extract_key_data(soup):
#     if not soup:
#         return None
# 
#     title = soup.find('title').text.strip() if soup.find('title') else 'N/A'
#     strain_name = soup.find('h2').text.strip() if soup.find('h2') else 'N/A'
#     synonyms = []
#     paragraphs = soup.find_all('p')
#     for p in paragraphs:
#         bold_text = p.find('b', string='Synonyms:')
#         if bold_text:
#             for content in p.contents:
#                 if content.name == 'a':
#                     synonyms.append(content.get_text(strip=True))
#                     synonyms.append('href: ' + content.get('href'))
#                 elif isinstance(content, str) and content.strip():
#                     synonyms.extend(content.split(', '))
#             break
# 
#     media_details = []
#     media_boxes = soup.find_all('div', class_='box')
#     for box in media_boxes:
#         media_title = box.find('h3', class_='title').text.strip() if box.find('h3', 'title') else 'N/A'
#         media_link = box.find('a', class_='link colorless')['href'] if box.find('a', 'link colorless') else 'N/A'
#         growth_observed = 'Yes' if box.find('i', class_='ph ph-lg ph-check text-success') else 'No'
#         growth_conditions = box.find('span', class_='badge danger').text.strip() if box.find('span',
#                                                                                              'badge danger') else 'N/A'
# 
#         media_details.append({
#             'media_title': media_title,
#             'media_link': media_link,
#             'growth_observed': growth_observed,
#             'growth_conditions': growth_conditions
#         })
# 
#     return {
#         'title': title,
#         'strain_name': strain_name,
#         'synonyms': synonyms,
#         'media_details': media_details
#     }
# 
# 
# # Function to extract strain details from a row of the DataFrame
# def extract_strain_details(row):
#     url = row['Name Link']
#     soup = fetch_html_structure(url)
#     return extract_key_data(soup)
# 
# 
# # Function to save detailed data incrementally
# def save_detailed_data(detailed_data):
#     df_detailed = pd.DataFrame(detailed_data)
#     df_detailed.to_csv('detailed_dsmz_data.csv', index=False)
# 
# 
# # Load the initial DataFrame (assuming it's loaded in variable df)
# # df = pd.read_csv('initial_data.csv')  # Load your initial DataFrame here
# 
# # Initialize an empty list to hold detailed data
# detailed_data = []
# num_strains = df.shape[0]
# max_workers = 64  # Adjust the number of workers as needed
# 
# # Extract detailed information for each strain with ThreadPoolExecutor
# with ThreadPoolExecutor(max_workers=max_workers) as executor:
#     future_to_index = {executor.submit(extract_strain_details, row): index for index, row in df.iterrows()}
# 
#     for future in tqdm(as_completed(future_to_index), total=num_strains, desc="Extracting strain details"):
#         index = future_to_index[future]
#         strain_details = future.result()
#         if strain_details:
#             detailed_data.append({
#                 "Name": df.loc[index, 'Name'],
#                 "Synonyms": ', '.join(strain_details['synonyms']),
#                 "Growth Conditions": strain_details['media_details']
#             })
#             # Save incrementally after each strain is processed
#             save_detailed_data(detailed_data)
# 
# # Sort the detailed data based on the Name to maintain order
# detailed_data.sort(key=lambda x: int(x['Name'].split(' ')[-1]))
# 
# # Create the final detailed DataFrame
# df_detailed = pd.DataFrame(detailed_data)
# df_detailed.to_csv('detailed_dsmz_data.csv', index=False)
# 
# # print("Detailed data saved to detailed_dsmz_data.csv")
# 
# # Load the detailed data
# df_detailed = pd.read_csv('detailed_dsmz_data.csv')
# 
# # Merge the initial and detailed DataFrames with progress bar
# merged_data = []
# for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Merging data"):
#     detailed_row = df_detailed[df_detailed['Name'] == row['Name']]
#     if not detailed_row.empty:
#         merged_row = {**row.to_dict(), **detailed_row.iloc[0].to_dict()}
#     else:
#         merged_row = row.to_dict()
#     merged_data.append(merged_row)
# 
# # Convert the merged data to a DataFrame and save it
# merged_data.sort(key=lambda x: int(x['Name'].split(' ')[-1]))
# merged_df = pd.DataFrame(merged_data)
# merged_df.to_csv('merged_dsmz_strains.csv', index=False)
# 
# # print("Merged data saved to merged_dsmz_strains.csv")